# Part 2: RNA-seq Differential Expression Analysis

**Welcome back!** In this notebook, we will take the clean, prepared data from Part 1 and perform a full differential gene expression analysis. This will allow us to identify which genes are changing their expression levels in the `hy5` mutant compared to the wild-type plants.

## 1. Setting Up Our Analysis Environment

We'll start by importing the necessary libraries for our statistical analysis and visualization.

In [ ]:
import pandas as pdimport numpy as npimport matplotlib.pyplot as pltfrom sklearn.decomposition import PCAfrom sklearn.preprocessing import StandardScalerfrom pydeseq2.dds import DeseqDataSetfrom pydeseq2.ds import DeseqStatsimport osimport reimport csv

## 2. Principal Component Analysis (PCA)

**Principal Component Analysis (PCA)** is a powerful statistical technique that simplifies complex data. Think of it as finding the most important directions (called **principal components**) in your data that capture the greatest amount of variation. When we apply this to our RNA-seq data, it helps us visualize our samples and understand the main factors driving gene expression differences, like the genotype.

In [ ]:
# Load data that was cleaned in Part 1
counts_df = pd.read_csv('gene_counts.txt', sep='\t', index_col=0, header=0, comment='#')
metadata_df = pd.read_csv('metadata.csv', index_col=0)

# Perform the same cleaning steps from Part 1
counts_df = counts_df.drop(columns=['Chr', 'Start', 'End', 'Strand', 'Length'])
summary_rows = ['Unassigned_Unmapped', 'Unassigned_Read_Type', 'Unassigned_Singleton', 'Unassigned_MappingQuality', 'Unassigned_Chimera', 'Unassigned_FragmentLength', 'Unassigned_Duplicate', 'Unassigned_MultiMapping', 'Unassigned_Secondary', 'Unassigned_NonSplit', 'Unassigned_NoFeatures', 'Unassigned_Overlapping_Length', 'Unassigned_Ambiguity']
counts_df = counts_df.loc[~counts_df.index.isin(summary_rows)]
counts_df.columns = [os.path.basename(col).replace('_Aligned.sortedByCoord.out.bam', '') for col in counts_df.columns]
counts_df = counts_df.T

# Ensure sample order matches between counts and metadata
if not all(counts_df.index == metadata_df.index):
    counts_df = counts_df.loc[metadata_df.index]

# Data normalization is crucial for PCA. We'll use log-transformation and then scale.
# Add a small pseudo-count to avoid log(0).
counts_normalized = np.log2(counts_df + 1)

# Standardize the data so each gene has a mean of 0 and a standard deviation of 1.
scaler = StandardScaler()
counts_scaled = scaler.fit_transform(counts_normalized)

# Perform PCA with 2 components for plotting
pca = PCA(n_components=2)
principal_components = pca.fit_transform(counts_scaled)

# Create a DataFrame for the principal components
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'], index=metadata_df.index)
pca_df['genotype'] = metadata_df['genotype']

# Calculate the explained variance ratio for each component
explained_variance = pca.explained_variance_ratio_

### Exercise: Interpreting the PCA Plot

Run the code below to generate the PCA plot. What do the results tell you about your samples? Do the replicates for each genotype cluster together? 

In [ ]:
# Set up the plot
fig, ax = plt.subplots(figsize=(8, 8))
genotypes = pca_df['genotype'].unique()

# Assign a color and marker for each genotype
colors = ['red', 'blue']
markers = ['o', 's']

# Plot the data points
for i, genotype in enumerate(genotypes):
    subset = pca_df[pca_df['genotype'] == genotype]
    ax.scatter(subset['PC1'], subset['PC2'],
               color=colors[i], marker=markers[i], s=100, label=genotype)

# Add labels and a title
ax.set_xlabel(f"Principal Component 1 ({explained_variance[0]*100:.2f}%)")
ax.set_ylabel(f"Principal Component 2 ({explained_variance[1]*100:.2f}%)")
ax.set_title("PCA of RNA-seq Samples")
ax.legend()
ax.grid(True, linestyle='--', alpha=0.6)

# Show the plot
plt.show()

---

## 3. Differential Expression Analysis with PyDESeq2

Now for the core of our analysis! We will use the `pydeseq2` library to identify genes that are significantly up- or down-regulated between the `hy5` mutant and the wild-type. 

In [ ]:
# Create the DeseqDataSet object
dds = DeseqDataSet(
    counts=counts_df,
    metadata=metadata_df,
    design='~genotype',
    refit_cooks=True
)

# Run the full PyDESeq2 pipeline
dds.deseq2()

# Get the results for the contrast 'hy5' vs 'Col-0'
stat_results = DeseqStats(dds, contrast=['genotype', 'hy5', 'Col-0'])
stat_results.run_wald_test()
stat_results.summary()
results_df = stat_results.results_df

# Save the results for advanced analysis later
results_df.to_csv('analysis_results.csv')

### DGE Explained: Significance and Fold Change

-   **Significance (Adjusted p-value):** The p-value is the probability that we would observe our data if there were truly no difference between the two groups. In DGE, we test thousands of genes, so we use the **adjusted p-value** (or FDR) to correct for this. We typically use a cutoff like `padj < 0.05` to identify genes with a statistically significant change in expression.

-   **Fold Change (log2FoldChange):** This value tells you the magnitude and direction of the change. A `log2FoldChange` of 1 means a gene is expressed 2 times more, and a value of -1 means it is expressed 2 times less. A value of 0 means no change. We often use a cutoff like `abs(log2FoldChange) > 1` to find biologically meaningful changes.

In [ ]:
# Your solution here

In [ ]:
# A possible solution
padj_cutoff = 0.05
lfc_cutoff = 1

significant_genes = results_df[
    (results_df['padj'] < padj_cutoff) & 
    (abs(results_df['log2FoldChange']) > lfc_cutoff)
]

print(f"Found {len(significant_genes)} significant genes.")

---

## 4. Advanced Analysis and Interpretation

### 4a. Annotating and Visualizing Top Genes

To make our results interpretable, we will first load our annotation file (`Arabidopsis_thaliana.TAIR10.61.gff3`), extract the gene names, and use this information to create a bar chart of the top 20 most significant genes. 

In [ ]:
def get_annotations_from_gff3(gff3_path):
    """Parses a GFF3 file and returns a DataFrame with gene annotations."""
    annotations = {}
    with open(gff3_path, 'r') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
            if row[0].startswith('#'):
                continue
            if len(row) > 2 and row[2] == 'gene':
                attributes = row[8]
                gene_id = None
                gene_symbol = None
                id_match = re.search(r'ID=(gene:.*?);', attributes)
                name_match = re.search(r'Name=(.*?);', attributes)
                if id_match:
                    gene_id = id_match.group(1)
                if name_match:
                    gene_symbol = name_match.group(1)
                if gene_id:
                    annotations[gene_id] = {'symbol': gene_symbol}
    return pd.DataFrame.from_dict(annotations, orient='index')

results_df = pd.read_csv('analysis_results.csv', index_col=0)
gff3_path = 'Arabidopsis_thaliana.TAIR10.61.gff3'
annotation_df = get_annotations_from_gff3(gff3_path)
annotated_significant_genes = results_df.join(annotation_df, how='left')

padj_cutoff = 0.05
lfc_cutoff = 1

top_upregulated = annotated_significant_genes[annotated_significant_genes['log2FoldChange'] > 0].head(10)
top_downregulated = annotated_significant_genes[annotated_significant_genes['log2FoldChange'] < 0].head(10)
top_genes_for_plot = pd.concat([top_upregulated, top_downregulated])

gene_labels = top_genes_for_plot['symbol'].fillna(pd.Series(top_genes_for_plot.index).str.replace('gene:', '')).tolist()

fig, ax = plt.subplots(figsize=(12, 8))
ax.barh(gene_labels, top_genes_for_plot['log2FoldChange'], color=['red' if fc > 0 else 'blue' for fc in top_genes_for_plot['log2FoldChange']])
ax.set_xlabel('Log2 Fold Change')
ax.set_title('Top 10 Up- and Downregulated Genes (hy5 vs Col-0)')
ax.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

### 4b. Functional Enrichment Analysis

Now that we have our lists of up- and downregulated genes, we want to understand their collective biological function. This process is called **Functional Enrichment Analysis**, which identifies if certain biological terms (e.g., pathways, functions) are overrepresented in your gene lists.

We will use the **PANTHER Classification System**, a free online tool, to perform this. You will use the lists of genes we created below to find enriched functions. Here's a quick guide on how to use the tool and interpret your results.

1.  **Prepare your gene lists**: The code below saves your significant genes to two text files, `upregulated_genes.txt` and `downregulated_genes.txt`.
2.  **Go to the PANTHER website**: Open the [PANTHER Overrepresentation Test](http://www.pantherdb.org/tools/upload.do) in your browser.
3.  **Upload your list**: Copy the contents of one of the `.txt` files and paste it into the query list box on the website. Be sure to select *Arabidopsis thaliana* as the organism.
4.  **Interpret the results**: The results table will show you which Gene Ontology (GO) terms are significantly enriched. Look for terms with a low p-value (e.g., `< 0.05`). The bar chart on the website will provide a nice visualization of these results. 

In [ ]:
upregulated_genes_list = annotated_significant_genes[annotated_significant_genes['log2FoldChange'] > 0].index.tolist()
downregulated_genes_list = annotated_significant_genes[annotated_significant_genes['log2FoldChange'] < 0].index.tolist()

with open('upregulated_genes.txt', 'w') as f:
    for gene in upregulated_genes_list:
        f.write(f"{gene}\n")

with open('downregulated_genes.txt', 'w') as f:
    for gene in downregulated_genes_list:
        f.write(f"{gene}\n")

print(f"Upregulated gene list saved to 'upregulated_genes.txt' ({len(upregulated_genes_list)} genes).")
print(f"Downregulated gene list saved to 'downregulated_genes.txt' ({len(downregulated_genes_list)} genes).")

---